In [11]:
import yaml
from pyprnt import prnt
import re

In [12]:
def get_between(subject, start, end, index=-1):
    matches = re.findall(f"{start}(.*){end}", subject)
    if index > -1:
        return matches[index] if len(matches) >= index+1 else None
    return matches if len(matches) else None


def rm_quotes(astr):
    if astr[0] in ("'", '"'):
        astr = astr[1:]
    if astr[len(astr)-1] in ("'", '"'):
        astr = astr[:-1]
    return astr


def extract_exports(bash_lines):
    _exports = [ get_between(e, 'export ', "\n", 0) for e in bash_lines if 'export' in e ]
    _exports = [ e.split('=') for e in _exports if isinstance(e, str) ]
    _exports = { e[0]: rm_quotes(e[1]) for k, e in enumerate(_exports) if isinstance(e, list) and len(e) == 2 and len(e[1]) > 0 }
    return _exports

def exports_from_file(file_path):
    with open(file_path, 'r') as _envrc:
        return extract_exports(_envrc.readlines())


def yaml_shape(_exports, _scope='RUN_TIME'):
    return {
          'envs': [
              {
                  'key': k,
                  'value': v,
                  'scope': 'RUN_TIME',
                  'type': 'SECRET' if v == '***' else 'GENERAL'
              }  for k, v in exports.items()
          ]
      }


def yaml_file_update_envs(new_yaml, save_path):
    with open(save_path, 'r') as appfile:
        if _yaml := yaml.safe_load(appfile.read()):
            _yaml.update(new_yaml)
            prnt(_yaml)
        else:
            _yaml = new_yaml
    with open(save_path, 'w') as appfile:
        appfile.write(yaml.safe_dump(_yaml))

In [13]:
exports = exports_from_file('../.envrc')
# Star-out some fields for security.
'''exports['MYSQL_PASSWORD'] = '***'
exports['CSRF_SECRET'] = '***'
exports['SMS_TWILIO_AUTH_TOKEN'] = '***'
exports['BAUTH_PASSWORD'] = '***'''

"exports['MYSQL_PASSWORD'] = '***'\nexports['CSRF_SECRET'] = '***'\nexports['SMS_TWILIO_AUTH_TOKEN'] = '***'\nexports['BAUTH_PASSWORD'] = '***"

In [14]:
exports_shaped = yaml_shape(exports)
yaml_file_update_envs(exports_shaped, '../.do/app.yaml')


╭─────────┬──────────────────────────────────────╮
│envs     │╭──┬─────────────────────────────────╮│
│         ││0 │╭─────┬────────────────────╮     ││
│         ││  ││key  │SITE_USE_MULTITHREAD│     ││
│         ││  ││value│False               │     ││
│         ││  ││scope│RUN_TIME            │     ││
│         ││  ││type │GENERAL             │     ││
│         ││  │╰─────┴────────────────────╯     ││
│         ││1 │╭─────┬─────────╮                ││
│         ││  ││key  │SITE_HOST│                ││
│         ││  ││value│0.0.0.0  │                ││
│         ││  ││scope│RUN_TIME │                ││
│         ││  ││type │GENERAL  │                ││
│         ││  │╰─────┴─────────╯                ││
│         ││2 │╭─────┬─────────╮                ││
│         ││  ││key  │SITE_PORT│                ││
│         ││  ││value│2095     │                ││
│         ││  ││scope│RUN_TIME │                ││
│         ││  ││type │GENERAL  │                ││
│         ││  │╰─────┴─────────